In [1]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [27]:
#計測地点のosm_idを指定
start_osm = 152781248
mid_osm = []
end_osm =  307601946
start_mesh = 50326612314
end_mesh = 50326612413

In [28]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [29]:
getdbname = "kddi_18_new_param"
dbname = "kddi_trkn18_monthly_anbun"
trknName = "trkn_18_grp"
sttime = "2022-04-01 18:00:00"

In [30]:
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                cur.execute(sql)
                connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        cur.execute(sql)
        connection.commit()
        cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)


33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値47.27222222244477距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値49.97777777806826距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値44.56666666666667距離0.0 km
50326612413
存在メッシュ：50326612413値52.68333333333333距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値49.97777777806826距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値46.977777777907555距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値48.55555555572495距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値45.4距離0.0 km
50326612413
存在メッシュ：50326612413値50.13333333333333距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値48.55555555572495距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値42.21111111133959距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値44.98888888918712距離0.2892176975888725 km
5032661

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値10.261111111121622距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値10.388888888902608距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値10.133333333333333距離0.0 km
50326612413
存在メッシュ：50326612413値10.516666666666667距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値10.388888888902608距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値8.45555555556561距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値8.577777777790901距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値8.333333333333334距離0.0 km
50326612413
存在メッシュ：50326612413値8.700000000000001距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値8.577777777790901距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値8.389272030677876距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値8.711877394670651距離0.28921769758

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値14.28888888890991距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値14.544444444471882距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値14.033333333333333距離0.0 km
50326612413
存在メッシュ：50326612413値14.799999999999999距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値14.544444444471882距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値11.70000000001371距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値11.86666666668456距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値11.533333333333333距離0.0 km
50326612413
存在メッシュ：50326612413値12.033333333333333距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値11.86666666668456距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値10.488888888934586距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値11.04444444450409距離0.28921769

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値18.300000000035645距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値18.733333333379857距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値17.866666666666667距離0.0 km
50326612413
存在メッシュ：50326612413値19.166666666666668距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値18.733333333379857距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値16.433333333336073距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値16.466666666670243距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値16.4距離0.0 km
50326612413
存在メッシュ：50326612413値16.499999999999996距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値16.466666666670243距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値14.8距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値14.8距離0.2892176975888725 km
50326612314
存在メッシュ：503266

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値21.655555555631413距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値22.57777777787679距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値20.733333333333334距離0.0 km
50326612413
存在メッシュ：50326612413値23.5距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値22.57777777787679距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値18.711111111150412距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値19.188888888940188距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値18.233333333333334距離0.0 km
50326612413
存在メッシュ：50326612413値19.666666666666668距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値19.188888888940188距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値18.300000000035645距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値18.733333333379857距離0.2892176975888725 km

...

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値24.222222222256953距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値24.644444444489775距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値23.8距離0.0 km
50326612413
存在メッシュ：50326612413値25.066666666666666距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値24.644444444489775距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値23.48888888900313距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値24.877777777926894距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値22.1距離0.0 km
50326612413
存在メッシュ：50326612413値26.266666666666666距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値24.877777777926894距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値19.577777777871912距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値20.722222222345096距離0.2892176975888725 km
50326612314


33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
50326612323 is kddi none!
50326612324
存在メッシュ：50326612324値30.73333333353017距離0.2892176975888725 km
50326612314
50326612314 is kddi none!
50326612413
存在メッシュ：50326612413値32.56666666666667距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値30.73333333353017距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値25.43333333346768距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値27.06666666684203距離0.2892176975888725 km
50326612314
50326612314 is kddi none!
50326612413
存在メッシュ：50326612413値28.7距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値27.06666666684203距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値24.733333333412844距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値25.700000000103785距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値23.766666666666666距離0.0 km
50326612413
存在メッシュ：50

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値36.67222222235977距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値38.34444444462398距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値35.0距離0.0 km
50326612413
存在メッシュ：50326612413値40.016666666666666距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値38.34444444462398距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
50326612323 is kddi none!
50326612324
存在メッシュ：50326612324値37.600000000209356距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値33.7距離0.0 km
50326612413
存在メッシュ：50326612413値39.55距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値37.600000000209356距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
50326612323 is kddi none!
50326612324
存在メッシュ：50326612324値36.000000000182524距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値32.6距離0.0 km
50326612413
存在メッシュ：50326612413値37.7距離0.4338265463600201

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値46.977777777907555距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値48.55555555572495距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値45.4距離0.0 km
50326612413
存在メッシュ：50326612413値50.13333333333333距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値48.55555555572495距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値42.21111111133959距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値44.98888888918712距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値39.43333333333333距離0.0 km
50326612413
存在メッシュ：50326612413値47.766666666666666距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値44.98888888918712距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値41.01666666685996距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値43.366666666918974距離0.2892176975888725 km
50326

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値8.45555555556561距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値8.577777777790901距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値8.333333333333334距離0.0 km
50326612413
存在メッシュ：50326612413値8.700000000000001距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値8.577777777790901距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値8.389272030677876距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値8.711877394670651距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値8.066666666666666距離0.0 km
50326612413
存在メッシュ：50326612413値9.03448275862069距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値8.711877394670651距離0.2892176975888725 km


In [31]:
connection.close()

In [171]:
#CreateGeojsonFile